In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from keras.models import model_from_json
from skimage.transform import resize
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
#     print(ds)
    print("Id", ds.PatientID)
#     print("Modality", ds.Modality)
    print("Study Description", ds.StudyDescription)
#     print("Patient's Sex", ds.PatientSex)
#     print("Patient's Age", ds.PatientAge)
#     print("Body Part Examined", ds.BodyPartExamined)
#     print("Patient Position", ds.PatientPosition)
#     print("Photometric Interpretation", ds.PhotometricInterpretation)
    
    # filtering: check DICOM header for image position, image type and body part
    # Body Part
    
    # Patient Position
    position = ds.PatientPosition
    if position not in ['PA', 'AP']:
        print('Image rejected: {}, PatientPosition: {}'.format(filename, position))
        return None
    
    # Modali
    mod = ds.Modality
    if mod != 'DX':
        print('Image rejected: {}, Modality: {}'.format(filename, mod))
        return None
    
    body_part = ds.BodyPartExamined
    if body_part not in ['CHEST', 'chest']:
        print('Image rejected: {}, Body part: {}'.format(filename, body_part))
        return None
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = img.copy()
    proc_img = resize(img, img_size)
    proc_img  = proc_img / 255
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    model.load_weights(weight_path)
    print("Model loaded", model)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred = model.predict(img, batch_size = 1, verbose = True)
    
    if pred[0][0] > thresh:
        prediction = 'Pneumonia present, {}'.format(pred[0][0])
    else:
        prediction = 'Pneumonia not present, {}'.format(pred[0][0])

    return prediction 

In [3]:
check_dicom("test1.dcm")
check_dicom("test2.dcm")
check_dicom("test3.dcm")
check_dicom("test4.dcm")
check_dicom("test5.dcm")
check_dicom("test6.dcm")

Load file test1.dcm ...
Id 2
Study Description No Finding
Load file test2.dcm ...
Id 1
Study Description Cardiomegaly
Load file test3.dcm ...
Id 61
Study Description Effusion
Load file test4.dcm ...
Id 2
Study Description No Finding
Image rejected: test4.dcm, Body part: RIBCAGE
Load file test5.dcm ...
Id 2
Study Description No Finding
Image rejected: test5.dcm, Modality: CT
Load file test6.dcm ...
Id 2
Study Description No Finding
Image rejected: test6.dcm, PatientPosition: XX


In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0# loads the mean image value they used during training preprocessing
img_std = 0# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.47 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img  = check_dicom(i)
    
    if img is None:
        continue
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Model loaded <keras.engine.sequential.Sequential object at 0x7f24642908d0>
Load file test1.dcm ...
Id 2
Study Description No Finding
1/1 [==============================] - 1s 847ms/step
Pneumonia present, 0.5363113284111023
Load file test2.dcm ...
Id 1
Study Description Cardiomegaly
1/1 [==============================] - 1s 570ms/step
Pneumonia present, 0.5365277528762817
Load file test3.dcm ...
Id 61
Study Description Effusion
1/1 [==============================] - 1s 548ms/step
Pneumonia present, 0.5363778471946716
Load file test4.dcm ...
Id 2
Study Description No Finding
Image rejected: test4.dcm, Body part: RIBCAGE
Load file test5.dcm ...
Id 2
Study Description No Finding
Image rejected: test5.dcm, Modality: CT
Load file test6.dcm ...
Id 2
Study Description No Finding
Image rejected: test6.dcm, PatientPosition: XX
